In [ ]:
from TransversalitySheaf import *
from ConleySheaf import *
from FlowSheaf import *
from pychomp import *


import matplotlib.pyplot as plt
from matplotlib import colors
from mpl_toolkits.mplot3d import Axes3D
import numpy as np

In [ ]:
#Swallowtail
def ST ( x, params):
    a,b,c = params
    return x**4 + a*x**2 + b*x + c
#Cusp catastrophe
def cusp ( x, params ):
    a,b = params
    return -x**3 + a*x + b

In [ ]:
ODE_model = lambda x, params: ST( x, params )

In [ ]:
a_min,a_max = -1.0, 1.0
b_min, b_max = -1.0, 1.0
c_min, c_max = -1.0, 1.0
base_bounds = [[a_min,a_max],[b_min,b_max],[c_min,c_max]]
base_boxes = [10,10,10]
base = GeometricCubicalComplex(base_bounds, base_boxes)
B= base.complex
a_vert = [a_min+i*base.delta[0] for i in range(base_boxes[0]+1)]
b_vert = [b_min+i*base.delta[1] for i in range(base_boxes[1]+1)]
c_vert = [c_min+i*base.delta[2] for i in range(base_boxes[2]+1)]

In [ ]:
bounds = [[-1.25,1.25]]
boxes = [10]
phase_specs = [bounds,boxes] 
num_samples = 1

In [ ]:
%%time
flow_sheaf = SheafFromVertices(TransversalitySheaf(base, ODE_model, phase_specs, num_samples))

In [ ]:
%%time
graded_sheaf = GradeFlowSheaf(flow_sheaf)

In [ ]:
%%time
conley_sheaf = ConleySheaf(graded_sheaf)

In [ ]:
conley_sheaf

In [ ]:
face_poset = base.face_poset

In [ ]:
#face_poset

In [ ]:
#DrawGradedComplex(GradedComplex(B,lambda x : x), Poset(face_poset))

In [ ]:
#Clustering Algorithm
#Pseudo-Code:
#for each edge in FP.edges
#  if mapping[edge] is iso:
#     add inverse edge
#strongly connected components
# localize = face_poset
# for (u,v) in face_poset.edges():
#     conley_map = conley_sheaf.mapping[(u,v)]
#     if conley_map.poset_iso(conley_sheaf.mapping[v],conley_sheaf.mapping[u]):
#         localize.add_edge(v,u)

In [ ]:
def InducedPoset( G, predicate ):
    result = DirectedAcyclicGraph()
    S = set([v for v in G.vertices() if predicate(v)])
    for v in S:
        result.add_vertex(v)
    for v in S:
        for u in G.children(v):
            if u in S and u != v:
                result.add_edge(v,u)
    return Poset(result)
def subposet_iso(poset_map, A, B):
    #A,B the subposets
    img = [poset_map(v) for v in A.vertices()]
    img_set = set(img)
    if not len(img)==len(img_set):
        return False
    if not img_set==B.vertices():
        return False
    return True

In [ ]:
#Clustering algorithm, with flag for changing localization criterion
reducedFlag = True
localize = face_poset
for (u,v) in face_poset.edges():
    if B.rightfringe(u) or B.rightfringe(v): continue
    conley_map = conley_sheaf.mapping[(u,v)]
    gc_u = conley_sheaf.mapping[u]
    gc_v = conley_sheaf.mapping[v]
    rc_u = InducedPoset(gc_u.poset, lambda v : v in gc_u.graded_complex.count() )
    rc_v = InducedPoset(gc_v.poset, lambda v : v in gc_v.graded_complex.count() )
    if reducedFlag:
        if subposet_iso(conley_map.poset_map, rc_v, rc_u):
            #print(v,u)
            localize.add_edge(v,u)
    else:
        if conley_map.poset_iso(conley_sheaf.mapping[v],conley_sheaf.mapping[u]):
            #print(v,u)
            localize.add_edge(v,u)

In [ ]:
#localize

In [ ]:
scc,mapping = CondensationGraph(face_poset.vertices(),lambda x : face_poset.adjacencies(x))

In [ ]:
scc

In [ ]:
#mapping

In [ ]:
graded_base = GradedComplex(base.complex, lambda x : mapping[x])

In [ ]:
# def poset_iso(gc_A,gc_B,gcMap):
#     poset_map = gcMap.poset_map
#     img = [poset_map(v) for v in gc_A.poset.vertices()]
#     img_set = set(img)
#     #Check injective
#     if not len(img)==len(img_set):
#         return False
#     #Check surjective
#     if not img_set==gc_B.poset.vertices():
#         return False
#     return True

In [ ]:
DrawGradedComplex(graded_base,Poset(scc))

In [ ]:
reduced_base = ConnectionMatrix(graded_base)

In [ ]:
DrawGradedComplex(reduced_base,Poset(scc))

In [ ]:
for base_cube in B:
    print(base_cube,B.cell_dim(base_cube),base.geometry(base_cube))

In [ ]:
mapping

In [ ]:
inv_mapping = {}
for target in set(mapping.values()):
    inv_mapping[target] = []
    for key in mapping.keys():
        if mapping[key]==target:
            inv_mapping[target] += [key]

In [ ]:
inv_mapping

In [ ]:
for v in inv_mapping.keys():
    print("Component: " + str(v))
    for base_cube in inv_mapping[v]:
        print(base_cube,B.cell_dim(base_cube),base.geometry(base_cube))

In [ ]:
# fig, ax = plt.subplots()
# plt.title('Parameter Space')

# plt.xlabel('a')
# plt.ylabel('b')
# plt.grid(True)
# plt.xticks(a_vert)
# plt.yticks(b_vert)
# #plt.grid()
# plt.xlim(a_min, a_max)
# plt.ylim(b_min, b_max)
# plt.show()

In [ ]:
data = np.zeros((base_boxes[1],base_boxes[0]))
data.shape

In [ ]:
slice_level = 2
for top_cell in B(B.dimension()):
    if B.rightfringe(top_cell): continue
    a,b,c = B.coordinates(top_cell)
    if c==slice_level:
        data[b,a] = mapping[top_cell]

In [ ]:
#data

In [ ]:
xmin,xmax = base.bounds[0]
ymin,ymax = base.bounds[1]
plt.imshow(data,extent=[xmin,xmax,ymin,ymax],origin='lower')
plt.colorbar()
#x = np.linspace(xmin, xmax, 100)
#y = np.linspace(ymin, ymax, 100)
#X, Y = np.meshgrid(x,y)
#F = 4*X**3-27*Y**2
#plt.contour(X,Y,F,[0],colors=('b'))

In [ ]:
xmin,xmax = base.bounds[0]
ymin,ymax = base.bounds[1]
zmin,zmax = base.bounds[2]

In [ ]:
x = np.linspace(xmin, xmax, 100)
y = np.linspace(ymin, ymax, 100)
X, Y = np.meshgrid(x,y)
F = 4*X**3-27*Y**2

In [ ]:
type(F)

In [ ]:
# for top_cell in B(B.dimension()):
#     print(B.coordinates(top_cell))

In [ ]:
x = np.linspace(xmin, xmax, base.boxes[0])
y = np.linspace(ymin, ymax, base.boxes[1])
X, Y = np.meshgrid(x, y)

Z1 = np.zeros([base.boxes[0],base.boxes[1]])
Z3 = np.zeros([base.boxes[0],base.boxes[1]])
Z7 = np.zeros([base.boxes[0],base.boxes[1]])

for top_cell in B(B.dimension()):
    if B.rightfringe(top_cell): continue
    a,b,c = B.coordinates(top_cell)
    if c==1:
        Z1[b,a] = mapping[top_cell]
    if c==3:
        Z3[b,a] = mapping[top_cell]+3
    if c==7:
        Z7[b,a] = mapping[top_cell]+7

In [ ]:
Z1

In [ ]:
levels = np.linspace(-1, 1, 40)
plt.contour(X,Y,.1*np.sin(3*X)*np.sin(5*Y),levels = .1*levels)

In [ ]:
fig = plt.figure()
ax = fig.gca(projection='3d')




#Z = np.sin(X)*np.sin(Y)

levels = np.linspace(-1, 1, 40)

ax.contourf(X, Y, Z1, zdir='z', levels=.1*levels)
#ax.contourf(X, Y, Z2, zdir='z', levels=3+.1*levels)
#ax.contourf(X, Y, Z3, zdir='z', levels=7+.1*levels)

#ax.legend()
ax.set_xlim3d(xmin, xmax)
ax.set_ylim3d(ymin, ymax)
ax.set_zlim3d(zmin, zmax)

#plt.show()

In [ ]:
fig = plt.figure()
ax = fig.gca(projection='3d')

x = np.linspace(0, 1, 100)
X, Y = np.meshgrid(x, x)
#Z = np.sin(X)*np.sin(Y)

levels = np.linspace(-1, 1, 40)

ax.contourf(X, Y, .1*np.sin(3*X)*np.sin(5*Y), zdir='z', levels=.1*levels)
#ax.contourf(X, Y, 3+.1*np.sin(5*X)*np.sin(8*Y), zdir='z', levels=3+.1*levels)
#ax.contourf(X, Y, 7+.1*np.sin(7*X)*np.sin(3*Y), zdir='z', levels=7+.1*levels)

#ax.legend()
ax.set_xlim3d(0, 1)
ax.set_ylim3d(0, 1)
ax.set_zlim3d(0, 10)

In [ ]:
.1*levels